In [5]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

The % symbol denotes a magic command in Jupyter. It ensures that pip is run in the same Python environment as the Jupyter kernel. This prevents issues that might arise if the pip installed packages into a different environment.

In [6]:
%pip install -U sentence-transformers
%pip install srsly
#%pip install triton - dont' need with mac m2
%pip install git+https://github.com/huggingface/transformers.git # install new huggingface transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/kc/jkd3yt1s0436pgb396mzmmn00000gn/T/pip-req-build-eqvuq34w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/kc/jkd3yt1s0436pgb396mzmmn00000gn/T/pip-req-build-eqvuq34w
  Resolved https://github.com/huggingface/transformers.git to commit 5fa35344755d8d9c29610b57d175efd03776ae9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/metal.html

import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
else:
    print("MPS backend not available. Falling back to CPU.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/metal.html
ERROR: Could not find a version that satisfies the requirement torchvision (from versions: none)
ERROR: No matching distribution found for torchvision
Note: you may need to restart the kernel to use updated packages.
Using device: mps


In [8]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("joe32140/ModernBERT-base-msmarco")
# Note - for my medical project, seems like maybe using a different model would be better
# also may need to retinker the sentence transformer part

# Run inference
sentences = [
    'what county is hayden in',
    "Hayden is a city in Kootenai County, Idaho, United States. Located in the northern portion of the state, just north of Coeur d'Alene, its population was 13,294 at the 2010 census.",
    "According to the United States Census Bureau, the city has a total area of 9.61 square miles (24.89 km2), of which 9.60 square miles (24.86 km2) is land and 0.01 square miles (0.03 km2) is water. It lies at the southwestern end of Hayden Lake, and the elevation of the city is 2,287 feet (697 m) above sea level. Hayden is located on U.S. Route 95 at the junction of Route 41. It is also four miles (6 km) north of Interstate 90 and Coeur d'Alene. The Coeur d'Alene airport is northwest of Hayden.",
]

embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 768]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

(3, 768)
torch.Size([3, 3])


In [9]:
#!pip install transformers[sentencepiece]
from transformers import pipeline

text = "Angela Merkel is a politician in Germany and leader of the CDU"
hypothesis_template = "This text is about {}" # Based on this interview, does the patient have {} sign or symptom
classes_verbalized = ["politics", "economy", "entertainment", "environment"]

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use mps:0


In [10]:
%%time

zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)

CPU times: user 272 ms, sys: 135 ms, total: 406 ms
Wall time: 2.69 s


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU',
 'labels': ['politics', 'economy', 'environment', 'entertainment'],
 'scores': [0.9994651079177856,
  0.0002065974404104054,
  0.0001747352071106434,
  0.0001535148185212165]}

In [11]:
!wget https://raw.githubusercontent.com/koaning/arxiv-frontpage/refs/heads/main/data/annot/new-dataset.jsonl

zsh:1: command not found: wget


In [12]:
import srsly
inputs = list(srsly.read_jsonl("new-dataset.jsonl"))
texts = [d["text"] for d in inputs]
labels = [d["cats"]["new-dataset"] for d in inputs]

In [13]:
%%time

outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

CPU times: user 1min 22s, sys: 13.6 s, total: 1min 36s
Wall time: 2min 8s


In [14]:
import numpy as np
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

np.float64(0.8990536277602523)

In [15]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/792M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Device set to use mps:0


In [16]:
%%time

zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)

CPU times: user 273 ms, sys: 85.5 ms, total: 359 ms
Wall time: 1.41 s


{'sequence': 'Angela Merkel is a politician in Germany and leader of the CDU',
 'labels': ['politics', 'economy', 'environment', 'entertainment'],
 'scores': [0.9994553327560425,
  0.0004048754053656012,
  0.00010866236698348075,
  3.11371150019113e-05]}

In [17]:
%%time
outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

CPU times: user 1min 28s, sys: 14.1 s, total: 1min 42s
Wall time: 1min 34s


In [18]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

np.float64(0.8940964398377648)

In [19]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-base-zeroshot-v2.0")

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/299M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Device set to use mps:0


In [20]:
%%time

outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

CPU times: user 1min 13s, sys: 10.7 s, total: 1min 23s
Wall time: 1min 17s


In [21]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

np.float64(0.9062640829202343)

TEST CODE FOR FEATURE EXTRACTION

In [ ]:
# Setup
SEED_GLOBAL = 42
import locale
locale.getpreferredencoding = lambda: "UTF-8"  # force UTF-8 

In [ ]:
%pip install -U sentence-transformers
%pip install srsly
%pip install git+https://github.com/huggingface/transformers.git # install new huggingface transformers
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/metal.html



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/kc/jkd3yt1s0436pgb396mzmmn00000gn/T/pip-req-build-hdvr4os6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/kc/jkd3yt1s0436pgb396mzmmn00000gn/T/pip-req-build-hdvr4os6
  Resolved https://github.com/huggingface/transformers.git to commit 234168c4dc837da10c594408f15807a1d6f3fab4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/metal.html
Note: you may need to restart the kernel to use updated packages.


In [ ]:

import os
import random

import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
else:
    print("MPS backend not available. Falling back to CPU.")

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import srsly


np.random.seed(SEED_GLOBAL)
torch.manual_seed(SEED_GLOBAL)
random.seed(SEED_GLOBAL)

import llm

"""
from datasets import load_dataset, concatenate_datasets
import re
import time

import tqdm
import torch
from torch.utils.data import DataLoader

import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import ClassLabel
from datasets import load_dataset, load_metric, Dataset, DatasetDict, concatenate_datasets, list_metrics

from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report

import gc
from accelerate.utils import release_memory

import wandb
import json
from datetime import datetime
import argparse

from mdutils import MdUtils

# suppress unnecessary tokenizer warning https://github.com/huggingface/transformers/issues/14285
transformers.logging.set_verbosity_error()

"""

Using device: mps


'\nfrom datasets import load_dataset, concatenate_datasets\nimport re\nimport time\n\nimport tqdm\nimport torch\nfrom torch.utils.data import DataLoader\n\nimport transformers\nfrom transformers import AutoModelForSequenceClassification, AutoTokenizer\nfrom transformers import TrainingArguments, Trainer\nfrom datasets import ClassLabel\nfrom datasets import load_dataset, load_metric, Dataset, DatasetDict, concatenate_datasets, list_metrics\n\nfrom sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report\n\nimport gc\nfrom accelerate.utils import release_memory\n\nimport wandb\nimport json\nfrom datetime import datetime\nimport argparse\n\nfrom mdutils import MdUtils\n\n# suppress unnecessary tokenizer warning https://github.com/huggingface/transformers/issues/14285\ntransformers.logging.set_verbosity_error()\n\n'

In [ ]:
import asyncio, llm

async def run():
    model = llm.get_async_model("llama3.2:latest")
    response = model.prompt("A short poem about tea")
    print(await response.text())

asyncio.run(run())

In [ ]:
# Get OpenAI key from BASH config

import re

# Path to your zsh configuration file
zshrc_path = os.path.expanduser("~/.zshrc")
print(zshrc_path)

# Read the zshrc file and extract the OPENAI_API_KEY
with open(zshrc_path, "r") as file:
    for line in file:
        # Look for the export line for OPENAI_API_KEY
        match = re.match(r"export OPENAI_API_KEY=['\"]?([^'\"]+)['\"]?", line)
        if match:
            openai_api_key = match.group(1)
            os.environ["OPENAI_API_KEY"] = openai_api_key
            break

# Verify that the key was successfully set
print("OPENAI_API_KEY loaded:", os.environ.get("OPENAI_API_KEY"))

/Users/reblocke/.zshrc
OPENAI_API_KEY loaded: None


In [ ]:
# verify torch backend
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

# [ ] TODO: troubleshoot - Compiling the model with `torch.compile` and using a `torch.mps` device is not supported. Falling back to non-compiled mode.

tensor([1.], device='mps:0')


In [ ]:
from pypdf import PdfReader

# Path to your PDF file
pdf_path = '/Users/reblocke/Research/dx_chat_entropy/Chatbot Transcripts/Intermtn MS4 1 Transcript.pdf'

# Initialize PDF reader
reader = PdfReader(pdf_path)

# Extract text from each page
all_text = ""
for page in reader.pages:
    all_text += page.extract_text()

# Output the text
print(all_text)

Patient Case
PATIENT DOOR CHART and Learner Instructions
- Patient Name: Bettie
- Age: 48
- Legal Sex: Female
- Chief Complaint: Chest pain
- Clinical Setting: Outpatient clinic
Vital Signs:
- Blood Pressure Reading: 130/80 mmHg
- Pulse Rate: 75 bpm
- Respiratory Rate: 16 breaths/min
- Temperature (Celsius): 37.0
- SpO2: 97%Learner Tasks:
Learner Tasks
1. Obtain an appropriately focused and detailed history based upon the chief complaint.
2. Perform a pertinent physical examination based upon the chief complaint.
3. Discuss your diagnostic impressions and next steps with the patient.
4. Place appropriate orders for the patient.
5. Review results with the patient and further next steps.
6. Answer any questions the patient may have to the best of your ability.Patient Case
Clinical Interview:
Doctor: can you describe the chest pain?
Patient: Chest pain. Tightness.
Doctor: location?
Patient: Under the middle of my chest.
Doctor: have you had this before?
Patient: Yes, for about 3 months.
D

In [ ]:

client = OpenAI(api_key=OPENAI_API_KEY)
md = MarkItDown(llm_client=client, llm_model="gpt-4o-2024-11-20")
supported_extensions = ('.pptx', '.docx', '.pdf', '.jpg', '.jpeg', '.png')
files_to_convert = [f for f in os.listdir('.') if f.lower().endswith(supported_extensions)]
for file in files_to_convert:
    print(f"\nConverting {file}...")
    try:
        md_file = os.path.splitext(file)[0] + '.md'
        result = md.convert(file)
        with open(md_file, 'w') as f:
            f.write(result.text_content)
        
        print(f"Successfully converted {file} to {md_file}")
    except Exception as e:
        print(f"Error converting {file}: {str(e)}")

print("\nAll conversions completed!")


In [ ]:
hypothesis_template = "This text is about {}" # Based on this interview, does the patient have {} sign or symptom
inputs = list(srsly.read_jsonl("new-dataset.jsonl"))
texts = [d["text"] for d in inputs]
labels = [d["cats"]["new-dataset"] for d in inputs]

In [ ]:
# Print the first 1000 entries of texts and labels
for i, (text, label) in enumerate(zip(texts[:1000], labels[:1000]), start=1):
    print(f"Entry {i}:")
    print(f"Text: {text}")
    print(f"Label: {label}")
    print("-" * 50)  # Separator for better readability

Entry 1:
Text: Our code and dataset is available here.
Label: 1
--------------------------------------------------
Entry 2:
Text: We will release the dataset and code to facilitate future endeavors.
Label: 1
--------------------------------------------------
Entry 3:
Text: We release our dataset for others to use and build on.
Label: 1
--------------------------------------------------
Entry 4:
Text: Our dataset is available online.
Label: 1
--------------------------------------------------
Entry 5:
Text: We release the generated dataset and used prompts to facilitate future research.
Label: 1
--------------------------------------------------
Entry 6:
Text: Code and dataset will be available.
Label: 1
--------------------------------------------------
Entry 7:
Text: We demonstrate the value of the created dataset by performing a quantitative and qualitative analysis on the models' results.
Label: 1
--------------------------------------------------
Entry 8:
Text: We train our model o

In [ ]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0")

Device set to use mps:0


In [ ]:
%%time
outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

Compiling the model with `torch.compile` and using a `torch.mps` device is not supported. Falling back to non-compiled mode.


CPU times: user 1min 30s, sys: 14.1 s, total: 1min 44s
Wall time: 1min 36s


In [ ]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

np.float64(0.8940964398377648)